# Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
    FROM sqlite_master
WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Overview of the Data

In [3]:
%%sql
SELECT *
    FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* name — the name of the country.
* area — the total land and sea area of the country.
* population — the country's population.
* population_growth — the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births a year per 1,000 people.
* death_rate — the country's death rate, or the number of death a year per 1,000 people.
* area — the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's water area in square kilometers.

## Summary Statistics

In [4]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


## Exploring Outliers in the Data

In [5]:
%%sql
SELECT *
    FROM facts
WHERE population = (SELECT MAX(population) FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


In [6]:
%%sql
SELECT *
    FROM facts
WHERE population = (SELECT MIN(population) FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


## Summary Statistics Revisited

In [7]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
WHERE population <> (SELECT MAX(population) FROM facts)

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [8]:
%%sql
SELECT AVG(population), AVG(area)
    FROM facts

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


## Filtering for Densely Populated Countries

In [9]:
%%sql
SELECT *
    FROM facts
WHERE population > (SELECT AVG(population) FROM facts)
AND area < (SELECT AVG(area) FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


## Largest Population and Highest Growth Rate

In [10]:
%%sql
SELECT name, population
    FROM facts
WHERE population <> (SELECT MAX(population) FROM facts)
ORDER BY population DESC
LIMIT 1

Done.


name,population
China,1367485388


In [11]:
%%sql
SELECT name, MAX(population_growth)
    FROM facts

Done.


name,MAX(population_growth)
South Sudan,4.02


## Land and Water Ratios

### Countries with more land than water ranked in descending order

In [13]:
%%sql
SELECT name, CAST(area_land AS FLOAT)/area AS land_ratio
    FROM facts
ORDER BY land_ratio DESC
LIMIT 5;

Done.


name,land_ratio
Afghanistan,1.0
Algeria,1.0
Andorra,1.0
Angola,1.0
Antigua and Barbuda,1.0


### Countries with more water than land ranked in descending order

In [14]:
%%sql
SELECT name, CAST(area_water AS FLOAT)/area AS water_ratio
    FROM facts
ORDER BY water_ratio DESC
LIMIT 5;

Done.


name,water_ratio
British Indian Ocean Territory,0.9988970588235294
Virgin Islands,0.818848167539267
Puerto Rico,0.35682691610470596
"Bahamas, The",0.2788184438040346
Guinea-Bissau,0.2215916955017301


## Highest Expected Growth in Population Next Year

In [17]:
%%sql
SELECT name, population * (population_growth/100) AS growth_next_year
    FROM facts
WHERE population <> (SELECT MAX(population) FROM facts)
ORDER BY growth_next_year DESC
LIMIT 5

Done.


name,growth_next_year
India,15270686.124799998
China,6153684.246
Nigeria,4448270.372
Pakistan,2906653.3662
Ethiopia,2874562.1691


## Higher Death Rate than Birth Rate

In [20]:
%%sql
SELECT name, birth_rate, death_rate
    FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC

Done.


name,birth_rate,death_rate
Ukraine,10.72,14.46
Bulgaria,8.92,14.44
Latvia,10.0,14.31
Lithuania,10.1,14.27
Russia,11.6,13.69
Serbia,9.08,13.66
Belarus,10.7,13.36
Hungary,9.16,12.73
Moldova,12.0,12.59
Estonia,10.51,12.4


## Highest Population/Area Ratio

In [21]:
%%sql
SELECT name, population/area AS population_density
    FROM facts
ORDER BY population_density DESC
LIMIT 5

Done.


name,population_density
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191


### Comparison to previous query

In [22]:
%%sql
SELECT name, population/area AS population_density
    FROM facts
WHERE population > (SELECT AVG(population) FROM facts)
AND area < (SELECT AVG(area) FROM facts)
LIMIT 5

Done.


name,population_density
Bangladesh,1138
Germany,226
Japan,335
Philippines,336
Thailand,132
